# Evaluation of three entity tasks

This recipe describes how to perform the evaluation of **named entity recognition (NER)**, **entity disambiguation (ED)** and **entity linking (EL)** output in different scenarios:

1. The user does have groundtruth data, where groundtruth data is the manually verified entity tags of entities found in a given text. In this case **quantitative evaluation** is possible.  
2. The user does not have groundtruth data, but they are willing to manually inspect the NER output in order to spot and flag errors, inconsistencies, hallucinations, etc. In th|is case, **qualitative evaluation** is necessary. As this process is time-consuming, it can be supported by in-notebook visualizations for quick data inspection.  

Note that the recipe only showcases a subset of the possible approaches, cf. [Variations and alternatives](#scrollTo=-riY-m5r7xv_&line=3&uniqifier=1).

### Rationale

These methods help the user assess the quality of **named entity recognition (NER)**, **entity disambiguation (ED)** and **entity linking (EL)** outputs. This is essential for any application, but especially when communicating with lay people, who often have reservations about new technologies.

The cookbook also allows for evaluation both in a situation in which data comes with ground truth labels (quantitative evaluation) and in a situation where data is not labeled (qualitative evaluation, a.k.a. eye-balling).

To run the quantitative evaluation with use the [`HIPE-scorer`](https://github.com/hipe-eval/HIPE-scorer), a set of Python scripts developed as part of the [HIPE shared task](https://hipe-eval.github.io/), focused on named entity processing of historical documents. As such, these scripts have certain requirements, for example when it comes to file naming or data format.

Output data format can be fed to application recipes for visualizing and analyzing errors, making the estimation of the performance easier also for lay people.

### Process overview

The evaluation module takes as input a tsv file where the first column is the token and the others are used to classify the token.

If the file includes gold labels, the user can perform the quantitative evaluation of the annotated test data. The process uses the following steps:

1. Installing the HIPE scorer
2. Downloading the evaluation data and ground truths
3. Reshape data to the format required by the scorer
4. Running the scorer and saving the results

If the file does not include gold labels, the cookbook returns a visualization of the annotation and gives the possibility to the user to give a free-text feedback about the annotation.

## 1. Evaluation preparation

The notebook cells in this section contain the defintion of functions that are used further down in the notebook. These cells **must be run** but you don't need to inspect them closely unless you want to modify the behaviour of this notebook.

First we install some standard Python libraries

In [1]:
import copy
from docopt import docopt
import json
import logging
import os
import regex
import sys
from termcolor import colored
sys.path.insert(0, '../tasks')
import utils

Next, we test if the HIPE-scorer is available. If it is not, we install it

In [2]:
BASE_DIR = os.getcwd()

if not os.path.exists("HIPE-scorer/clef_evaluation.py"):
    ! git clone https://github.com/enriching-digital-heritage/HIPE-scorer.git
    os.chdir(os.path.join(BASE_DIR, "HIPE-scorer"))
    ! pip install -r requirements.txt
    ! pip install .
    os.chdir(BASE_DIR)

Finally, we load the HIPE-scorer

In [3]:
os.chdir(os.path.join(BASE_DIR, "HIPE-scorer"))
import clef_evaluation
os.chdir(BASE_DIR)

## 2. Evaluation of named entity recognition

Evaluation of named entity recognition performed by a large language model (LLM) is not trivial because current LLMs are not able to give back the exact position of the recognized entities. Therefore we first need to find the recognized entities back in the texts (function `get_char_offsets_of_entities`) before we can compare them with the correct gold standard data. For the comparison we use externaly developed software: the [HIPE-scorer](https://github.com/hipe-eval/HIPE-scorer). We need 12 helper functions for performing the evaluation.

In [4]:
def get_char_offsets_of_entity(text_cleaned, entity_text):
    """lookup entity in original text and return list of character offsets"""
    matches = regex.finditer(entity_text, text_cleaned)
    char_offsets = []
    for match in matches:
        char_offsets.append({"start_char": match.start(), "end_char": match.end()})
    return char_offsets

In [5]:
def get_char_offsets_of_entities(text_id, text, raw_entities):
    """lookup list of entities in original text and return list with character offsets"""
    cleaned_entities = []
    for raw_entity in raw_entities:
        char_offsets = get_char_offsets_of_entity(text, raw_entity["text"])
        if not char_offsets:
            print(f"raw entity not found in text! text_id: {text_id}; entity: {raw_entity['text']}")
            continue
        cleaned_entities.extend([char_offset | {"text": raw_entity["text"], "label": raw_entity["label"]} 
                               for char_offset in char_offsets])
    return cleaned_entities

In [6]:
def remove_overlapping_entities(entities):
    """remove overlapping entities from list of entities sorted by start offset"""
    indices_to_be_deleted = []
    for entity_index, entity in enumerate(entities):
        if entity_index > 0:
            if entities[entity_index - 1]["end_char"] > entities[entity_index]["start_char"]:
                indices_to_be_deleted.append(entity_index)
    for entity_index in reversed(indices_to_be_deleted):
        entities.pop(entity_index)
    return entities

In [7]:
def read_lines_from_file(file_name):
    """Read lines from the file file_name, return as list of strings"""
    if type(file_name) != str:
        return read_lines_from_stdin()
    else:
        with open(file_name, "r") as file_handle:
            lines = [line.strip() for line in file_handle]
        file_handle.close()
        return lines

In [8]:
def add_cleaned_entities_to_tokens(text_tokens, cleaned_entities, text_id):
    """Add entity information to token information and return the combination"""
    token_id = 0
    tokens_with_machine_tags = []
    for entity in cleaned_entities:
        if entity["label"] == "PLACE":
            entity["label"] = "LOCATION"
        if entity["label"] not in ["LOCATION", "PERSON"]:
            continue
        while token_id < len(text_tokens) and text_tokens[token_id]["start"] < entity["start_char"]:
            tokens_with_machine_tags.append(text_tokens[token_id] | {"machine_tag": "O"})
            token_id += 1
        if token_id < len(text_tokens):
            if text_tokens[token_id]["start"] != entity["start_char"]:
                print(f"clean entity not found in text! text_id: {text_id}; entity: {entity}; token: {text_tokens[token_id]}")
            else:
                iob = "B"
                while token_id < len(text_tokens) and text_tokens[token_id]["end"] <= entity["end_char"]:
                    tokens_with_machine_tags.append(text_tokens[token_id] | {"machine_tag": f"{iob}-{entity['label']}"})
                    token_id += 1
                    iob = "I"
                if abs(text_tokens[token_id - 1]["end"] - entity["end_char"]) > 1:
                    print(f"warning: machine token boundary error: text_id = {text_id}; entity: {entity}; token: {text_tokens[token_id - 1]}")
    while token_id < len(text_tokens):
        tokens_with_machine_tags.append(text_tokens[token_id] | {"machine_tag": "O"})
        token_id += 1
    return tokens_with_machine_tags

In [9]:
def read_machine_data(file_name):
    """Read the output file of the NER task and return the contents as a list of text results"""
    ner_json = utils.read_json_file(file_name)
    text_tokens = []
    for text_id, text in enumerate(ner_json):
        cleaned_entities = get_char_offsets_of_entities(text_id, text["text_cleaned"], text["entities"])
        tokens_with_machine_tags = add_cleaned_entities_to_tokens(text["tokens"],
                                                                  remove_overlapping_entities(sorted(cleaned_entities, 
                                                                                                     key=lambda entity: (entity["start_char"], 
                                                                                                                         -entity["end_char"]))),
                                                                  text_id)
        text_tokens.append(tokens_with_machine_tags)
    return text_tokens

In [10]:
def add_gold_entities_to_tokens(lines, text_tokens):
    """Add the golden standard entities to the text tokens"""
    text_tokens_text_index = 0
    text_tokens_token_index = 0
    last_label = "O"
    for line in lines:
        if line == "" or line == f"{DOC_SEPARATOR} {DOC_SEPARATOR}":
            continue
        label, token = line.split()
        if token != text_tokens[text_tokens_text_index][text_tokens_token_index]["text"]:
            print(f"token mismatch!: annotation: {token}; machine: {text_tokens[text_tokens_text_index][text_tokens_token_index]['text']}")
            break
        elif label not in ["p", "l"]:
            text_tokens[text_tokens_text_index][text_tokens_token_index]["gold_tag"] = "O"
            last_label = label
        else:
            iob = "I" if last_label == label else "B"
            text_tokens[text_tokens_text_index][text_tokens_token_index]["gold_tag"] = f"{iob}-{label_names[label]}"
            last_label = label
        text_tokens_token_index += 1
        if text_tokens_token_index >= len(text_tokens[text_tokens_text_index]):
            text_tokens_text_index += 1
            text_tokens_token_index = 0
    return text_tokens

In [11]:
HIPE_MACHINE_FILE = "HIPE_machine.txt"
HIPE_GOLD_FILE = "HIPE_gold.txt"


def save_data_for_evaluation(text_tokens):
    """Save the data for evaluation to two files: on for the machine analysis and one for the golden standard"""
    save_data_to_file(text_tokens, "machine_tag", HIPE_MACHINE_FILE)
    save_data_to_file(text_tokens, "gold_tag", HIPE_GOLD_FILE)


def save_data_to_file(text_tokens, data_column_name, file_name):
    """Save the data for evaluation to a single specified file"""
    with open(file_name, "w") as hipe_file:
        print("TOKEN\tNE-COARSE-LIT\tNE-COARSE-METO\tMISC", file=hipe_file)
        for text in text_tokens:
            sent_id = 0
            for token in text:
                if token["sent_id"] != sent_id:
                    print("-\tO\tO\tO", file=hipe_file)
                    sent_id = token["sent_id"]
                print(token["text"], token[data_column_name], token[data_column_name], "O", sep="\t", file=hipe_file)
            print("-\tO\tO\tO", file=hipe_file)
        hipe_file.close()

In [12]:
ARGS_VALUES = {'--glue': None, '--help': False, '--hipe_edition': 'hipe-2020',
 '--log': 'log.txt', '--n_best': '1', '--noise-level': None,
 '--original_nel': False, '--skip-check': False, '--suffix': None,
 '--tagset': None, '--time-period': None}


def run_scorer(args):
    """Run the HIPE-scorer to perform the evaluation"""
    tasks = ("nerc_coarse", "nel")
    if args["--task"] not in tasks:
        msg = "Please restrict to one of the available evaluation tasks: " + ", ".join(tasks)
        logging.error(msg)
        sys.exit(1)
    logging.debug(f"ARGUMENTS {args}")
    clef_evaluation.main(args=args | ARGS_VALUES)

In [13]:
SCORER_DIR = "HIPE-scorer"

def evaluate(text_tokens):
    """Evaluate the data present in text tokens: save them, run the scorer and show the results"""
    save_data_for_evaluation(text_tokens)
    os.chdir(os.path.join(BASE_DIR, SCORER_DIR))
    run_scorer(args={"--ref": os.path.join("..", HIPE_GOLD_FILE),
                     "--pred": os.path.join("..", HIPE_MACHINE_FILE),
                     "--task": "nerc_coarse",
                     "--outdir": "."})
    os.chdir(BASE_DIR)
    show_scores()

In [14]:
LABELS = ["PERSON", "LOCATION"]

def show_scores(labels=LABELS):
    """Extract relevant evaluation scores from the output file of the scorer and show them"""
    hipe_scores = utils.read_json_file(os.path.join(SCORER_DIR, HIPE_MACHINE_FILE))
    print(colored("HIPE Analysis", attrs=["bold"]))
    for label in labels:
        print(f"{label:>8}:", {key: round(100*value, 1) 
              for key, value in hipe_scores['NE-COARSE-LIT']['TIME-ALL']['LED-ALL'][label]['exact'].items() 
              if key in ['P_micro', 'R_micro', 'F1_micro']})

Below are the main commands for the named entity recognition evaluation:

1. Read the output of the machine
2. Read the golden standard annotations
3. Align the machine output with the annotations
4. Perform the evaluation with the HIPE-scorer and show the results

We evaluate the analysis from the NER file "output_ner_5f12fa7c16d33ea378148197569f999f774f7481.json". Change the name of this file to one of your own to analyse your data.

In [15]:
NER_OUTPUT_FILE = "../tasks/output_ner_5f12fa7c16d33ea378148197569f999f774f7481.json"
NER_ANNOTATIONS_FILE = "Wikimedia-random-100-annotations.txt"
DOC_SEPARATOR = "-DOCSTART-"
label_names = {"p": "PERSON", "l": "LOCATION"}

text_tokens = read_machine_data(NER_OUTPUT_FILE)
gold_lines = read_lines_from_file(NER_ANNOTATIONS_FILE)
text_tokens =  add_gold_entities_to_tokens(gold_lines, text_tokens)
evaluate(text_tokens)

true: 1 pred: 1
data_format_true [[3790]]
data_format_pred [[3790]]
HIPE Analysis
  PERSON: {'P_micro': 88.5, 'R_micro': 78.0, 'F1_micro': 82.9}
LOCATION: {'P_micro': 53.4, 'R_micro': 78.0, 'F1_micro': 63.4}


## 3. Evaluation of named entity linking

In [16]:
def add_gold_linking_entities_to_text_tokens(text_tokens, gold_entities):
    text_tokens_text_id = 0
    text_tokens_token_id = 0
    for gold_entities_text in gold_entities:
        for gold_entity in gold_entities_text:
            while text_tokens[text_tokens_text_id][text_tokens_token_id]["start"] < gold_entity["start"]:
                text_tokens[text_tokens_text_id][text_tokens_token_id]["wikidata_id"] = "_"
                text_tokens_token_id += 1
                if text_tokens_token_id >= len(text_tokens[text_tokens_text_id]):
                    break
            while text_tokens_token_id < len(text_tokens[text_tokens_text_id]) and text_tokens[text_tokens_text_id][text_tokens_token_id]["end"] <= gold_entity["end"]:
                if "wikidata_id" not in gold_entity:
                    text_tokens[text_tokens_text_id][text_tokens_token_id]["wikidata_id"] = "_"
                else:
                   text_tokens[text_tokens_text_id][text_tokens_token_id]["wikidata_id"] = gold_entity["wikidata_id"]["id"]
                text_tokens_token_id += 1
        while text_tokens_token_id < len(text_tokens[text_tokens_text_id]):
            text_tokens[text_tokens_text_id][text_tokens_token_id]["wikidata_id"] = "_"
            text_tokens_token_id += 1
        text_tokens_text_id += 1
        text_tokens_token_id = 0
    return text_tokens

In [17]:
def get_char_offsets_of_entities_linking(text_id, text, raw_entities):
    """lookup list of entities in original text and return list with character offsets"""
    cleaned_entities = []
    for raw_entity in raw_entities:
        char_offsets = get_char_offsets_of_entity(text, raw_entity["text"])
        if not char_offsets:
            print(f"raw entity not found in text! text_id: {text_id}; entity: {raw_entity['text']}")
            continue
        cleaned_entities.extend([char_offset | {"text": raw_entity["text"], "wikidata_id": raw_entity["wikidata_id"]} 
                               for char_offset in char_offsets])
    return cleaned_entities

In [18]:
def get_machine_entities(file_name):
    linking_json = utils.read_json_file(LINKING_OUTPUT_FILE)
    machine_entities = []
    for text_id, text in enumerate(linking_json):
        cleaned_entities = get_char_offsets_of_entities_linking(text_id, text["text_cleaned"], text["entities"])
        cleaned_entities = remove_overlapping_entities(sorted(cleaned_entities, 
                                                              key=lambda entity: (entity["start_char"], 
                                                                                  -entity["end_char"])))
        for cleaned_entity in cleaned_entities:
            cleaned_entity["start"] = cleaned_entity.pop("start_char")
            cleaned_entity["end"] = cleaned_entity.pop("end_char")
        machine_entities.append(cleaned_entities)
    return machine_entities

In [19]:
HIPE_MACHINE_FILE = "HIPE_machine.txt"
HIPE_GOLD_FILE = "HIPE_gold.txt"

def save_linking_data_to_file(text_tokens, data_column_name, file_name):
    """Save the data for evaluation to a single specified file"""
    with open(file_name, "w") as hipe_file:
        print("TOKEN\tNE-COARSE-LIT\tNE-COARSE-METO\tNEL-LIT\tNEL-METO\tMISC", file=hipe_file)
        for text in text_tokens:
            sent_id = 0
            for token in text:
                if token["sent_id"] != sent_id:
                    print("-\tO\tO\t_\t_\tO", file=hipe_file)
                    sent_id = token["sent_id"]
                print(token["text"], token[data_column_name], token[data_column_name], 
                                     token["wikidata_id"], token["wikidata_id"], "O", sep="\t", file=hipe_file)
            print("-\tO\tO\t_\t_\tO", file=hipe_file)
        hipe_file.close()

In [20]:
SCORER_DIR = "HIPE-scorer"

def evaluate_linking(text_tokens_gold, text_tokens_machine):
    """Evaluate the data present in text tokens: save them, run the scorer and show the results"""
    save_linking_data_to_file(text_tokens_gold, "gold_tag", HIPE_GOLD_FILE)
    save_linking_data_to_file(text_tokens_machine, "machine_tag", HIPE_MACHINE_FILE)
    os.chdir(os.path.join(BASE_DIR, SCORER_DIR))
    run_scorer(args={"--ref": os.path.join("..", HIPE_GOLD_FILE),
                     "--pred": os.path.join("..", HIPE_MACHINE_FILE),
                     "--task": "nel",
                     "--outdir": "."})
    os.chdir(BASE_DIR)
    show_scores_linking()

In [21]:
def show_scores_linking(labels=["ALL"]):
    """Extract relevant evaluation scores from the output file of the scorer and show them"""
    hipe_scores = utils.read_json_file(os.path.join(SCORER_DIR, HIPE_MACHINE_FILE))
    print(colored("HIPE Analysis", attrs=["bold"]))
    for label in labels:
        print(f"{label}:", {key: round(100*value, 1) 
              for key, value in hipe_scores['1']['NEL-LIT']['TIME-ALL']['LED-ALL'][label]['exact'].items() 
              if regex.search("micro", key)})

In [22]:
LINKING_OUTPUT_FILE = "../tasks/output_linking_34e26bfd19c837e400a5fcb214cd1e7a25304a12.json"

gold_entities = utils.read_json_file("linking_gold.json")
text_tokens_gold = add_gold_linking_entities_to_text_tokens(copy.deepcopy(text_tokens), gold_entities)

machine_entities = get_machine_entities(LINKING_OUTPUT_FILE)
text_tokens_machine = add_gold_linking_entities_to_text_tokens(copy.deepcopy(text_tokens), machine_entities)

evaluate_linking(text_tokens_gold, text_tokens_machine)

true: 1 pred: 1
data_format_true [[3790]]
data_format_pred [[3790]]
HIPE Analysis
ALL: {'P_micro': 62.1, 'R_micro': 78.6, 'F1_micro': 69.4}


## 4. Compare analysis with gold data

We need helper functions for accessing the HIPE-scorer, for converting the data to the scorer format and for interpreting the analysis of the scorer

In [ ]:
def get_label_value(entity, label):
    if label not in entity:
        return ""
    else:
        return entity[label][list(entity[label].keys())[0]]

In [ ]:
PRED_FILE_TASKS = "../tasks/output_linking_34e26bfd19c837e400a5fcb214cd1e7a25304a12.json"

with open(os.path.join(BASE_DIR, PRED_FILE_TASKS), "r") as infile:
    pred_data = json.load(infile)
    infile.close()

all_tokens = []
for text in pred_data:
    tokens = {token["start"]: {"text": token["text"], "label": "O"}
              for token in text["tokens"]}
    next_id = -1
    for char_id in reversed(tokens.keys()):
        tokens[char_id]["next_id"] = next_id
        next_id = char_id
    for entity in text["entities"]:
        try:
            wikidata_id = get_label_value(entity, "wikidata_id")
            link = get_label_value(entity, "link")
            tokens[entity["start_char"]]["label"] = "B-" + entity["label"]
            tokens[entity["start_char"]]["wikidata_id"] = wikidata_id
            tokens[entity["start_char"]]["link"] = link 
            next_id = tokens[entity["start_char"]]["next_id"]
            while next_id >= 0 and next_id < entity["end_char"]:
                tokens[next_id]["label"] = "I-" + entity["label"]
                tokens[next_id]["wikidata_id"] = wikidata_id
                tokens[next_id]["link"] = link
                next_id = tokens[next_id]["next_id"]
        except Exception as e:
            print(f"warning: no token at character index {e}")
    all_tokens.append(tokens)

In [ ]:
all_tokens[9]

In [ ]:
PRED_FILE = "example.tsv"
GOLD_FILE = "example.tsv"
SCORER_DIR = "HIPE-scorer"
os.chdir(os.path.join(BASE_DIR, "HIPE-scorer"))
run_scorer(args={"--ref": GOLD_FILE,
                 "--pred": PRED_FILE,
                 "--task": "nerc_coarse",
                 "--outdir": "RESULT_FOLDER"})
os.chdir(BASE_DIR)

## 5. Preparation for manual assessment

In [ ]:
import pandas as pd
from IPython.display import HTML, display

# Function for visualising the entities with link

def highlight_entities(
    data,
    iob_column = "NE-COARSE-LIT",
    base_url="https://www.wikidata.org/wiki/"
    ):
    # 1) Rebuild the text with spacing rules
    text_parts = []
    for idx, row in data.iterrows():
        tok = row["TOKEN"]
        if "NoSpaceAfter" in row["MISC"]:
            text_parts.append(tok)
        else:
            text_parts.append(tok + " ")
    text = "".join(text_parts)

    # 2) Merge contiguous IOB entities of the same type
    entities = []
    current = None  # {"start_char": int, "end_char": int, "label": str}

    for idx, row in data.iterrows():
        tag = row[iob_column]
        if tag == "_" or tag == "O":
            # close any open entity
            if current is not None:
                entities.append(current)
                current = None
            continue

        # Extract type and prefix
        if tag.startswith("B-"):
            etype = tag[2:]
            eid = row["NEL-LIT"]
            # close previous if open
            if current is not None:
                entities.append(current)
            # start new
            current = {
                "start_char": int(row["start_char"]),
                "end_char": int(row["end_char"]),
                "label": etype,
                "id": eid
            }

        elif tag.startswith("I-"):
            etype = tag[2:]
            eid = row["NEL-LIT"]
            if current is not None and current["label"] == etype:
                # extend current run
                current["end_char"] = int(row["end_char"])
            else:
                # stray I- (no open run or different type) → treat as B-
                if current is not None:
                    entities.append(current)
                current = {
                    "start_char": int(row["start_char"]),
                    "end_char": int(row["end_char"]),
                    "label": etype,
                    "id": eid
                }

        else:
            # Unknown tag → close any open entity
            if current is not None:
                entities.append(current)
                current = None

    # flush any remaining entity
    if current is not None:
        entities.append(current)

    # 3) Render with spans (note: end_char is inclusive → slice to en+1)
    entities.sort(key=lambda e: e["start_char"])
    result = ""
    last_idx = 0

    for e in entities:
        s, en = int(e["start_char"]), int(e["end_char"])
        etext = text[s:en + 1]  # inclusive end
        etype = e.get("label", "Other")
        eid = e.get("id", "")
        color = label_to_color.get(etype, "#dddddd")

        # decide whether to show eid as link or not
        if eid !="_" and eid !="NIL":
          eid_html = f'<a href="{base_url}{eid}">{eid}</a>'
        else:
          eid_html = ""  # if entity linking was not successful no link is shown

        result += text[last_idx:s]
        result += (
            f'<span style="background-color:{color}; color:black; padding:3px 6px; '
            f'border-radius:16px; margin:0 2px; display:inline-block; '
            f'box-shadow: 1px 1px 3px rgba(0,0,0,0.1);">'
            f'{etext}'
            f'<span style="font-size:0.75em; font-weight:normal; margin-left:6px; '
            f'background-color:rgba(0,0,0,0.05); padding:1px 6px; border-radius:12px;">'
            f'{etype} {eid_html}</span></span>'
        )
        last_idx = en + 1  # continue after inclusive end

    result += text[last_idx:]
    return result,text,entities

## 6. Data preparation

In [ ]:
# we need a function to convert the British Museum groundtruth data into the format expected by the HIPE scorer.
# The data is found here: http://145.38.185.232/enriching/bm.txt

In [ ]:
def get_demo_data():
  import pandas as pd
  # Loading the data
  data = pd.read_csv(
      'https://raw.githubusercontent.com/wjbmattingly/llm-lod-recipes/refs/heads/main/output/sample.tsv',
      sep='\t'
  )

  # Adding the start and end characters per token to the dataframe
  data['start_char'] = 0
  data['end_char'] = 0
  current_char = 0

  for index, row in data.iterrows():
      data.loc[index, 'start_char'] = current_char
      token = row['TOKEN']
      # Check if the next token should not have a space before it
      if index + 1 < len(data) and 'NoSpaceAfter' in data.loc[index, 'MISC']:
          current_char += len(token)
      else:
          current_char += len(token) + 1  # Add 1 for the space after the token

      data.loc[index, 'end_char'] = current_char - 1 # Subtract 1 because end_char is inclusive

  # Just for testing purposes: adds a Wikidata ID to one entity
  data.loc[0, 'NEL-LIT'] = 'Q1744'
  return data

In [ ]:
df = get_demo_data()

In [ ]:
# preparing data for groundtruth evaluation
import regex as re
import csv

def clean_format(input_file,output_file):
  file = open(input_file)
  output = open(output_file,mode='w')
  reader = file.readlines() #(file,delimiter="\t")
  #writer = csv.writer(output,delimiter="\t")
  output.write('\t'.join(["TOKEN","NE-COARSE-LIT","NE-COARSE-METO","NE-FINE-LIT","NE-FINE-METO","NE-FINE-COMP","NE-NESTED","NEL-LIT","NEL-METO","MISC\n"]))
  i = 1
  for line in reader:
    line = line.strip()
    mod_line = re.sub('_','-',line)
    if re.search('-DOCSTART- -DOCSTART- -DOCSTART-',mod_line):
      mod_line = re.sub('-DOCSTART- -DOCSTART- -DOCSTART-',f'# document_{i}',mod_line)
      i+=1
    mod_line = mod_line.split(' ')
    try:
      if len(mod_line)==2:
        output.write("\n"+" ".join(mod_line)+"\n")
      else:
        output.write('\t'.join([mod_line[0],mod_line[1],'-','-','-','-','-',mod_line[2],'-','-\n',]))
    except: continue

In [ ]:
clean_format('./data/bm_labels.txt','./data/gold/sample.tsv')
clean_format('./data/bm-2-ner-format.txt','./data/predictions/sample.tsv')
#

## 7. NER evaluation with groundtruth data by using the HIPE scorer

In [ ]:
! python clef_evaluation.py --help

### 7.1 Running the scorer

 ‼️ In the cell below it is important to note the parameter `--task`. This parameter value needs to be adjusted depending on the task one wants to evaluate (i.e. NER or EL). When evaluating NER we use `--task nerc_coarse`, while for evaluating EL we use `--task nel`.

In [ ]:
import glob
import regex as re

for doc in glob.glob('./data/predictions/*'):
  gold = re.sub('predictions','gold',doc)
  ! python clef_evaluation.py --ref "{gold}" --pred "{doc}" --task nerc_coarse --outdir ./data/evaluations/ --hipe_edition hipe-2020 --log=./data/evaluations/scorer.log


Let's now look at the various bits of data produced by the scorer. They are in the folder specified in the `--outdir` parameter of the scorer.

In [ ]:
ls -la ./evaluation_data/evaluations/

Here is an overview of the files created by the scorer:
- `scorer.log` – the log produced by the scorer
- `01_sample_nerc_coarse.tsv` – a TSV file contaning the evaluation results for document `01_sample` and task `nerc_coarse`, at different levels of aggregation etc.
- `01_sample_nerc_coarse.json` – a JSON file with a more granular breakdown of the evaluation, which can be useful for error analysis and to better understand systems' performance.

In [ ]:
! cat ./evaluation_data/evaluations/scorer.log

In [ ]:
import pandas as pd

eval_df = pd.read_csv('./evaluation_data/evaluations/01_sample_nerc_coarse.tsv', sep='\t')
eval_df.drop(columns=['System'], inplace=True)
eval_df.set_index('Evaluation', inplace=True)

Let's print the **micro-averaged** precision, recall and F-score in a **strict** evaluation regime:

In [ ]:
eval_df.loc['NE-COARSE-LIT-micro-strict-TIME-ALL-LED-ALL'][['P', 'R', 'F1']]

Let's print the **micro-averaged** precision, recall and F-score in a **fuzzy** evaluation regime:

In [ ]:
# Let's print the micro-averaged precision, recall and F-score in a *fuzzy* evaluation regime
eval_df.loc['NE-COARSE-LIT-micro-fuzzy-TIME-ALL-LED-ALL'][['P', 'R', 'F1']]

There is more data in the TSV file, as it can be seen when printing the whole content:

In [ ]:
! cat ./evaluation_data/evaluations/01_sample_nerc_coarse.tsv

## 8. Manual assessment of the output

### 8.1. Displaying the entities

To allow for manual assessment of output quality, the following cells display the identified entities in form of color-coded annotations via HTML. These kinds of insights into the results can both complement quantitative statistics and work as another way to estimate output quality. The latter is especially important for the frequent cases where no gold (or silver or bronze) standard is available that the NER output can be evaluated on. (Also, note that there are also other tools or modules out there that provide similar visualisations, e.g. [displaCy](https://spacy.io/usage/visualizers#ent) when using spaCy for NER.)

In [ ]:
data = get_demo_data()
data.head()

In [ ]:
# Highlighting the identified entities in form of color-coded annotations with links to authority files where available

# Color palette - add more colors if more labels are used or change them here
colors = ['#F0D9EF', '#FCDCE1', '#FFE6BB', '#E9ECCE', '#CDE9DC', '#C4DFE5', '#D9E5F0', '#F0E6D9', '#E0D9F0', '#E6FFF0', '#9CC5DF']

# Name Labels that should be shown in color, not mentioned labels will be shown in grey (this makes it easier to focus on certain categories if needed)
labels = ["PERSON", "DATE"]

# Mapping each label from the label set to a color from the palette
label_to_color = {label: colors[i % len(colors)] for i, label in enumerate(labels)}

# Generating the HTML - two changes can be made here:
# 1) by default, the column "NE-COARSE-LIT" is used for the entities, this can be changed via the argument "iob_column"
# 2) the entity identifiers are taken from the column "NEL-LIT"; by default, these are expected to be Wikidata identifiers (e.g. Q1744) and are combined with the Wikidate base URL; for another authority file, the base URL can be changed via the argument "base_url"
res,text,entities = highlight_entities(data)

# displaying the annotations
display(HTML(res))

### 8.2. Giving feedback on the entities

The following cell gives a very simple example for how manual assessment of entities could be integrated into the data. Here, the user is asked for feedback on each identified entity which then shows up in designated column.

In [ ]:
# Create a new column for manual assessment
data['manual_assessment'] = ''

# Display results again for better overview (no scrolling back and front)
display(HTML(res))

# Iterate through the identified entities
for e in entities:
    s, en = int(e["start_char"]), int(e["end_char"])
    etext = text[s:en + 1]
    etype = e.get("label", "Other")

    # Ask for feedback on the entity
    feedback = input(f'Is the entity "{etext.strip()}" with label "{etype}" correct? (y/n/feedback): ')

    # Store the feedback for all tokens within the entity span
    for index, row in data.iterrows():
        token_start = int(row["start_char"])
        token_end = int(row["end_char"])
        if max(s, token_start) <= min(en, token_end):
            data.loc[index, 'manual_assessment'] = feedback

# Show the altered data (now with user assessments)
data

## 9. EL evaluation with groundtruth data

TODO.

## 10. Variations and alternatives

Another approach if the user does not have groundtruth data could be to use an **LLM-judge** approach to evaluate the NER output in absence of labelled golden data. The task of the LLM is then to “review” the NER output and to assess its quality.